In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import os

# ============================================
# 🔹 Hàm tiện ích lấy an toàn
# ============================================
def safe_find(driver, selector, attr="text", timeout=10):
    """Lấy phần tử an toàn"""
    try:
        el = WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))
        return el.text.strip() if attr == "text" else el.get_attribute(attr)
    except:
        return ""

# ============================================
# 🔹 Crawl thông tin từ 1 URL Google Maps
# ============================================
def crawl_place_info(url):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless=new")  # Bỏ nếu muốn thấy Chrome
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(options=options)
    driver.get(url)
    time.sleep(3)

    # --- Tên địa điểm ---
    name = safe_find(driver, ".DUwDvf.lfPIob")

    # --- Địa chỉ (từ aria-label) ---
    address = ""
    try:
        address_elem = driver.find_element(By.CSS_SELECTOR, 'button[data-item-id="address"]')
        address = address_elem.get_attribute("aria-label").replace("Address:", "").strip()
    except:
        pass

    # --- Số điện thoại (từ aria-label) ---
    phone = ""
    try:
        phone_elem = driver.find_element(By.CSS_SELECTOR, 'button[data-item-id^="phone:"]')
        phone = phone_elem.get_attribute("aria-label").replace("Phone:", "").strip()
    except:
        pass

    driver.quit()

    # In kết quả ra console
    print(f"📍 {name}")
    print(f"🏠 {address}")
    print(f"📞 {phone}")
    print(f"🔗 {url}")
    print("-" * 60)

    return {
        "Tên": name,
        "Địa chỉ": address,
        "Số điện thoại": phone,
        "Link": url
    }

# ============================================
# 🔹 Chạy danh sách link từ CSV
# ============================================
def crawl_from_csv(input_file, output_file="maps_data.csv", save_every=100):
    df = pd.read_csv(input_file)
    col_name = "hfpxzc_href" if "hfpxzc_href" in df.columns else "Link"

    links = df[col_name].dropna().unique()
    print(f"🔍 Bắt đầu crawl {len(links)} địa điểm...\n")

    results = []
    # Nếu đã có file trước đó, load để nối tiếp
    if os.path.exists(output_file):
        existing = pd.read_csv(output_file)
        done_links = set(existing["Link"])
        links = [l for l in links if l not in done_links]
        results = existing.to_dict("records")
        print(f"⚡ Tiếp tục từ file cũ, đã có {len(done_links)} link, còn lại {len(links)} cần crawl.\n")

    for i, link in enumerate(links, start=1):
        try:
            print(f"Đang crawl {i}/{len(links)}: {link}")
            data = crawl_place_info(link)
            results.append(data)
            print(f"Đã lấy: {data['Tên']} - {data['Số điện thoại']}\n")
        except Exception as e:
            print(f"Lỗi ở {link}: {e}")
            results.append({"Tên": "", "Địa chỉ": "", "Số điện thoại": "", "Link": link})

        # 🔹 Lưu định kỳ sau mỗi N bản ghi
        if i % save_every == 0:
            pd.DataFrame(results).to_csv(output_file, index=False, encoding="utf-8-sig")
            print(f"Đã lưu tạm sau {i} dòng → {output_file}\n")

    # 🔹 Lưu cuối cùng
    pd.DataFrame(results).to_csv(output_file, index=False, encoding="utf-8-sig")
    print(f"\ Hoàn tất! Dữ liệu lưu tại: {output_file}")

# ============================================
# 🔹 Chạy chương trình
# ============================================
if __name__ == "__main__":
    crawl_from_csv("href_links.csv", output_file="maps_data.csv", save_every=100)


FileNotFoundError: [Errno 2] No such file or directory: 'href_links.csv'